In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Original data set http://people.sc.fsu.edu/~jburkardt/datasets/regression/x03.txt

In [ ]:
!cat data.txt

In [ ]:
with open('data.txt','r') as file:
    original = file.readlines()
    print(original)
    l = [e.strip('\n').strip().split(' ') for e in original if e[0] is not '#']
    print(l)

In [ ]:
df = pd.DataFrame(l)

In [ ]:
df

In [ ]:
df = df.drop([0,1,3,5],axis=1).drop(range(6))

In [ ]:
df = df.reset_index().drop('index',axis=1)

In [ ]:
df

In [ ]:
df.columns=['intercept','age','bp']

In [ ]:
df['age']

In [ ]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

In [ ]:
df.head()

In [ ]:
x=df['age'].values
y=df['bp'].values

In [ ]:
def plot_lr(x,y,beta0=None,beta1=None):
    plt.figure()
    plt.xticks(np.arange(min(x), max(x)+1, 5))
    plt.yticks(np.arange(min(y), max(y)+1, 10))
    plt.xlabel('Age')
    plt.ylabel('Blood Pressure')
    plt.scatter(x,y)
    if beta0 and beta1:
        plt.plot(x,beta0+beta1*x,c='red')
        plt.title('Coefficients: beta0 = {b0:.{d}f}, beta1 = {b1:.{d}f}'.format(b0=beta0, b1=beta1, d=1));
    

In [ ]:
plot_lr(x,y)

# Linear Regression: Ordinary Least Squares

In [ ]:
# Normal Equations

X = df[['intercept','age']].values

In [ ]:
W1 = np.linalg.inv(X.T.dot(X))
W2 = np.matmul(X.T,y)
W = np.matmul(W1, W2)

In [ ]:
W

In [ ]:
beta0 = W[0]
beta1 = W[1]

In [ ]:
plot_lr(x,y,beta0,beta1)

# Gradient Descent

#### Hyperparameters: Arbitrarily chosen by user before training

In [ ]:
beta_init = np.array([0.1, 0.2]) # initialize the weights
alpha = 10**-4 # set a learning rate

**Gradient Descent algorithm**

Mean Squared Error

$MSE =  \frac{1}{m}\sum_i^m (y_i - \hat{y}_i)^2 = \frac{1}{m}\sum_i^m (y_i - X_i\hat{\beta} )^2$

Gradient w.r.t. $\beta$

$\nabla_{beta} = \frac{2}{m}\sum_i^n X^T(y_i - X_i\hat{\beta} )$



Update rule

$\beta_{new} = \beta_{old} - \alpha \nabla_{beta}$    i.e. step in the negative gradient direcation

In [ ]:
def calc_cost(X,y,beta):
    mse = 1/m*sum((y-X.dot(beta))**2)
    return(mse)

# Plot Cost function

In [ ]:
beta_test = np.array([np.ones(1000),np.linspace(-100,100,1000)])

In [ ]:
m = len(y)

In [ ]:
costs = np.zeros(1000)
beta1s = np.linspace(-10,12,1000)
beta0_test = 100
for i in range(1000):
    costs[i]=calc_cost(X,y,np.array([beta0_test,beta1s[i]]))

plt.title('Cost function for different model parameters')
plt.plot(beta1s,costs);

In [ ]:
calc_cost(X,y,beta_init)

In [ ]:
#n = len(y)

In [ ]:
iterations = range(300000)
cost_history = np.zeros(len(iterations))

beta = beta_init

for i in iterations:
    cost_history[i] = calc_cost(X,y,beta)
    gradient = 2/m*X.T.dot(X.dot(beta)-y)
    beta = beta - alpha *gradient

In [ ]:
beta

In [ ]:
plt.title('Cost against iteration')
plt.plot(range(4,300000), cost_history[4:300000]);

In [ ]:
plot_lr(x,y,beta0=beta[0],beta1=beta[1])

# Singular Matrix w Gradient Descent

In [ ]:
X2 = np.hstack([X,X[:,1].reshape(-1,1)])

In [ ]:
X2

In [ ]:
def gd(X,y,beta=beta_init):
    iterations = range(300000)
    cost_history = np.zeros(len(iterations))

    for i in iterations:
        cost_history[i] = calc_cost(X,y,beta)
        gradient = 2/n*X.T.dot(X.dot(beta)-y)
        beta = beta - alpha *gradient
    return(beta)

In [ ]:
gd(X2,y,beta=np.array([0.1,.1,.1]))

In [ ]:
np.linalg.inv(X2.T.dot(X2))

In [ ]:
# However, Moore-Penrose pseudo inverse works!
np.linalg.pinv(X2.T.dot(X2))

In [ ]:
# pseudo inverse
W1 = np.linalg.pinv(X2.T.dot(X2))
W2 = X2.T.dot(y)
W1.dot(W2)

# Experiment with the Sigmoid function model parameters

In [ ]:
beta0 = 0
beta1 = 1
x = np.linspace(-10,10,100)

In [ ]:
t=beta0+beta1*x

In [ ]:
sig = 1/(1+np.exp(-t))

In [ ]:
plt.grid()
plt.title('Sigmoid Function with beta0={} and beta1={}'.format(beta0,beta1))
plt.yticks(np.linspace(0,1,11))
plt.plot(x,sig);

**Note:** Cut off, where sig = 0.5 is $\frac{-\beta_0}{\beta_1}$

# Logistic Regression Example with scikit-learn

In [ ]:
from sklearn import datasets

In [ ]:
data = datasets.load_iris()

In [ ]:
data.keys()

In [ ]:
X = data['data']

In [ ]:
X_df = pd.DataFrame(X[:,2:], columns=data['feature_names'][2:])

In [ ]:
y=data['target']

In [ ]:
data['target_names']

In [ ]:
y = (y==2).astype(int) # if Virginica

In [ ]:
y_df = pd.DataFrame(y)

In [ ]:
X_df

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression(C=400) #instantiate

In [ ]:
logreg.fit(X,y) # fit model

In [ ]:
print('Accuracy on training set: '+str(np.round(sum(logreg.predict(X)==y)/len(y)*100,1))+'%')